In [9]:
import pandas as pd 

cesta_souboru = r'C:\Users\petrrozkosny\Downloads\2024_03_Seznam-operaci_List-of-Operations_14_20.xlsx'
df = pd.read_excel(cesta_souboru,skiprows=2)
df = df.drop(0,axis=0)
df = df[['Program','IČ','PSČ příjemce','Projekt s právním aktem o poskytnutí / převodu podpor','Datum zahájení fyzické operace',\
         'Skutečné datum ukončení fyzické realizace operace','Kód NUTS 3','Celkové způsobilé výdaje přidělené na operaci (CZK)',\
         'Finanční prostředky vyúčtované v žádostech o platbu (celkové způsobilé výdaje, CZK)'
         
         ]]

df.rename(columns={'Celkové způsobilé výdaje přidělené na operaci (CZK)':'CZV','Finanční prostředky vyúčtované v žádostech o platbu (celkové způsobilé výdaje, CZK)':'ZOP',\
                   'Datum zahájení fyzické operace':'Zahájení','Skutečné datum ukončení fyzické realizace operace': 'Ukončení',\
                   'Projekt s právním aktem o poskytnutí / převodu podpor':'PA',\
                   'PSČ příjemce':'PSČ'},inplace=True)
df.dropna(subset=['Zahájení','Ukončení'],inplace=True)
df['Zahájení'] = pd.to_datetime(df['Zahájení'],errors='coerce')
df['Ukončení'] = pd.to_datetime(df['Ukončení'],errors='coerce')

C:\Users\petrrozkosny\AppData\Local\Temp\ipykernel_15480\1849528650.py:17: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Zahájení'] = pd.to_datetime(df['Zahájení'],errors='coerce')
C:\Users\petrrozkosny\AppData\Local\Temp\ipykernel_15480\1849528650.py:18: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Ukončení'] = pd.to_datetime(df['Ukončení'],errors='coerce')


In [3]:
df['CZV'] = df['CZV'].astype('float')
df['ZOP'] = df['ZOP'].astype('float')
df['PA'] = pd.to_datetime(df['PA'])
df['Zahájení'] = pd.to_datetime(df['Zahájení'])
df['Ukončení'] = pd.to_datetime(df['Ukončení'])



In [4]:
df['Realizace'] = df['Ukončení'] - df['Zahájení']
df['Nedocerpano'] = df['CZV'] - df['ZOP']

df_group = df.copy()

df_group = df.groupby(by=['IČ','Program']).agg({'CZV':'sum','ZOP':'sum','Realizace':'mean','Nedocerpano':'sum','IČ':'count'})
df_group

CZV          ZOP          Realizace  Nedocerpano  IČ
IČ       Program                                                              
00000078 OP ZP     4568006.85   4568006.75  718 days 00:00:00         0.10   2
00000175 OP ZP     3005281.39   3005281.39  979 days 00:00:00         0.00   2
00000205 IROP     67298150.00  55524967.91 1947 days 00:00:00  11773182.09   1
         OP PIK    2971180.00   2680226.09  328 days 00:00:00    290953.91   1
         OP ZP    20243688.07  19200342.18  899 days 13:20:00   1043345.89   9
...                       ...          ...                ...          ...  ..
88521931 OP PIK     550414.00    550414.00  278 days 00:00:00         0.00   1
88592472 OP PIK   84100000.00  84086932.48  486 days 00:00:00     13067.52   2
88610411 OP Z       784756.25    784756.24 1248 days 00:00:00         0.01   1
88675955 OP PIK    5300000.00   5300000.00  403 days 00:00:00         0.00   1
88687708 OP Z      2490756.00   2462400.00  729 days 00:00:00     28356.00   1

[33331 rows x 5 columns]

In [5]:
df_group2 = df.groupby(by=['Program']).agg({'IČ':'count','Realizace':'mean'})
df_group2['Realizace'] = df_group2['Realizace'].dt.days
df_group2



IČ  Realizace
Program                  
IROP     12537        816
OP D       302        809
OP PIK   13443        535
OP PPR    1576        675
OP TP      271        806
OP VVV   20926        760
OP Z     10009        829
OP ZP     6821        602

In [16]:
import ipywidgets as widgets    

output = widgets.Output()


df = df.loc[df['Zahájení'].notnull()]
df['Zahájení ROK'] = df['Zahájení'].dt.year
df['Zahájení ROK'] = df['Zahájení ROK'].astype('int64')
df.sort_values(by='Zahájení ROK',ascending=True,inplace=True)


list_rok = df['Zahájení ROK'].unique()  
widget_rok = widgets.Dropdown(options=list_rok,description='Rok:',disabled=False)#
#widget_text = widgets.Text(value='Zde bude zobrazen výpis',disabled=False)

def filtruj(change):
    with output:
        output.clear_output()

        display(df[df['Zahájení ROK'] == change.new])


widget_rok.observe(filtruj,names='value')
display(widget_rok,output)  

Dropdown(description='Rok:', options=(1956, 2010, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, …

Output()